# Project1-ResNet_Ablation_Study

#### 목표

1) ResNet-34, ResNet-50 모델 구현해보기 <br/>
2) 구현한 ResNet을 통한 Image Classification 모델 훈련 <br/>
3) Ablation Study 결과 만들어보기 <br/>


# ResNet-34, ResNet-50 모델 구현해보기

#### 라이브러리 실행

In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds

# graphviz로 시각화 하기
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

%matplotlib inline

## 1. 34 레이어 구성하기

### 1) ResNet 34


#### ResNet 34 Block 구성

https://d2l.ai/chapter_convolutional-modern/resnet.html

In [2]:
def ResNet34_Block(input_layer,
                 channel=64,
#                  stage_num=1,
                 first=False):
    
    # 입력 레이어
    x = input_layer
    
    if not first:
    
        # Conv2D
        x = keras.layers.Conv2D(
            filters=channel,
            kernel_size=(3,3),
            kernel_initializer='he_normal',
            padding='same'
#             name=f'stage{stage_num}_conv1'
        )(x)
    
        # Batch Normalization 
        x = keras.layers.BatchNormalization(
#             name=f'stage{stage_num}_batch1'
        )(x)

        # Activation
        x = keras.layers.Activation('relu')(x)

        # Conv2D
        x = keras.layers.Conv2D(
            filters=channel,
            kernel_size=(3,3),
            kernel_initializer='he_normal',
            padding='same'
#             name=f'stage{stage_num}_conv2'
        )(x)

        # Batch Normalization 
        x = keras.layers.BatchNormalization(
#             name=f'stage{stage_num}_batch2'
        )(x)

        # Add
        x = keras.layers.Add()([input_layer, x])

        # Activation
        x = keras.layers.Activation('relu')(x)
    
    else:
        
        # Conv2D
        x = keras.layers.Conv2D(
            filters=channel,
            kernel_size=(3,3),
            strides=2,
            kernel_initializer='he_normal',
            padding='same'
#             name=f'stage{stage_num}_conv1'
        )(x)
    
        # Batch Normalization 
        x = keras.layers.BatchNormalization(
#             name=f'stage{stage_num}_batch1'
        )(x)

        # Activation
        x = keras.layers.Activation('relu')(x)

        # Conv2D
        x = keras.layers.Conv2D(
            filters=channel,
            kernel_size=(3,3),
            kernel_initializer='he_normal',
            padding='same'
#             name=f'stage{stage_num}_conv1'
        )(x)
    
        # Batch Normalization 
        x = keras.layers.BatchNormalization(
#             name=f'stage{stage_num}_batch1'
        )(x)        
                
        # Conv2D
        y = keras.layers.Conv2D(
            filters=channel,
            kernel_size=(1,1),
            strides=2,
            kernel_initializer='he_normal',
            padding='same'
#             name=f'stage{stage_num}_conv2'
        )(input_layer)

        # Batch Normalization 
        y = keras.layers.BatchNormalization(
#             name=f'stage{stage_num}_batch2'
        )(y)

        # Add
        x = keras.layers.Add()([x, y])

        # Activation
        x = keras.layers.Activation('relu')(x)        
    
    return x

#### ResNet 34 구성

In [3]:
def resnet_34(input_shape=(32,32,3),
              channel_list=[64,64,64,128,128,128,128,256,256,256,256,256,256,512,512,512],
              first_list=[False,False,False,True,False,False,False,True,False,False,False,False,False,True,False,False],
              num_classes=2):
    
    input_layer = keras.layers.Input(shape=input_shape)  # input layer를 만들어둡니다.
        

    # Conv1
    x = keras.layers.Conv2D(
        filters=64,
        kernel_size=(7,7),
        strides=2,
        kernel_initializer='he_normal',
        padding='same'
    )(input_layer)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = keras.layers.MaxPooling2D(
             pool_size=(2, 2),
             strides=2
    )(x)


    
    for i, (channel, first) in enumerate(zip(channel_list, first_list)): 
        x = ResNet34_Block(
            x,
            channel=channel,
            first = first
        )
    
    output = keras.layers.AveragePooling2D(
             pool_size=(2, 2),
             strides=2,
             padding='same'
             
    )(x)
    
    
    output = keras.layers.Flatten()(output)
    output = keras.layers.Dense(num_classes, activation='softmax')(output)

    
    model = keras.Model(
            inputs = input_layer,
            outputs = output)

    
    return model

### 2) Plain 34 

#### Plain 34 Block 

In [4]:
def plain_34_Block(input_layer,
                   num_cnn=2, 
                   channel=64,
                   block_num=1,
                   stride=1
                   ):
    # 입력 레이어
    x = input_layer
    
    for cnn_num in range(num_cnn):
        # Conv2D
        x = keras.layers.Conv2D(
            filters=channel,
            kernel_size=(3,3),
            kernel_initializer='he_normal',
            padding='same',
            strides=stride,
            name=f'block{block_num}_conv{cnn_num}'
        )(x)
    
        # Batch Normalization 
        x = keras.layers.BatchNormalization(
            name=f'block{block_num}_batch{cnn_num}'
        )(x)

        # Activation
        x = keras.layers.Activation('relu',
            name=f'block{block_num}_act{cnn_num}'
        )(x)
        
    return x


#### Plain34 구성

In [5]:
def plain_34(input_shape=(32,32,3),
             num_cnn_list=[3,4,6,3],
             channel_list=[64,128,256,512],
             num_classes=2,
             stride=1
             ):
    
    input_layer = keras.layers.Input(shape=input_shape)  # input layer를 만들어둡니다.
        

    # Conv1
    x = keras.layers.Conv2D(
        filters=64,
        kernel_size=(7,7),
        strides=2,
        kernel_initializer='he_normal',
        padding='same'
    )(input_layer)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = keras.layers.MaxPooling2D(
             pool_size=(2, 2),
             strides=2
    )(x)


    
    for i, (num_cnn, channel) in enumerate(zip(num_cnn_list, channel_list)): 
        
        if i == 1:
            stride=2
        else:
            stride=1
                
        x = plain_34_Block(
            x,
            num_cnn=num_cnn, 
            channel=channel,
            block_num=i,
            stride=stride
        )

        
    output = keras.layers.AveragePooling2D(
             pool_size=(2, 2),
             strides=2,
             padding='same'
             
    )(x)
    
    
    output = keras.layers.Flatten()(output)
    output = keras.layers.Dense(num_classes, activation='softmax')(output)

    
    model = keras.Model(
            inputs = input_layer,
            outputs = output)

    
    return model

## 2. 50 layer 구성하기

### 1) ResNet 50 구성

#### ResNet 50 Block 구성

In [6]:
def ResNet50_Block(input_layer,
                 channel=64,
#                  stage_num=1,
                 first=False):
    
    # 입력 레이어
    x = input_layer
    
    if not first:
    
        # Conv2D
        x = keras.layers.Conv2D(
            filters=channel,
            kernel_size=(1,1),
            kernel_initializer='he_normal',
            padding='same'
#             name=f'stage{stage_num}_conv1'
        )(x)
    
        # Batch Normalization 
        x = keras.layers.BatchNormalization(
#             name=f'stage{stage_num}_batch1'
        )(x)

        # Activation
        x = keras.layers.Activation('relu')(x)

        # Conv2D
        x = keras.layers.Conv2D(
            filters=channel,
            kernel_size=(3,3),
            kernel_initializer='he_normal',
            padding='same'
#             name=f'stage{stage_num}_conv1'
        )(x)
    
        # Batch Normalization 
        x = keras.layers.BatchNormalization(
#             name=f'stage{stage_num}_batch1'
        )(x)

        # Activation
        x = keras.layers.Activation('relu')(x)        

        # Conv2D
        x = keras.layers.Conv2D(
            filters=channel*4,
            kernel_size=(1,1),
            kernel_initializer='he_normal',
            padding='same'
#             name=f'stage{stage_num}_conv2'
        )(x)

        # Batch Normalization 
        x = keras.layers.BatchNormalization(
#             name=f'stage{stage_num}_batch2'
        )(x)
        
        # Add
        x = keras.layers.Add()([x, input_layer])

        # Activation
        x = keras.layers.Activation('relu')(x)
    
    else:
        
        if channel == 64: 
            stride=1 
        else: 
            stride=2
            # Conv2D
        x = keras.layers.Conv2D(
            filters=channel,
            kernel_size=(1,1),
            strides=stride,
            kernel_initializer='he_normal',
            padding='same'
#           name=f'stage{stage_num}_conv1'
        )(x)

        # Batch Normalization 
        x = keras.layers.BatchNormalization(
#             name=f'stage{stage_num}_batch1'
        )(x)

        # Activation
        x = keras.layers.Activation('relu')(x)

        # Conv2D
        x = keras.layers.Conv2D(
            filters=channel,
            kernel_size=(3,3),
            kernel_initializer='he_normal',
            padding='same'
#             name=f'stage{stage_num}_conv1'
        )(x)

        # Batch Normalization 
        x = keras.layers.BatchNormalization(
#             name=f'stage{stage_num}_batch1'
        )(x)

        # Activation
        x = keras.layers.Activation('relu')(x)        

        # Conv2D
        x = keras.layers.Conv2D(
            filters=channel*4,
            kernel_size=(1,1),
            kernel_initializer='he_normal',
            padding='same'
#             name=f'stage{stage_num}_conv2'
        )(x)

        # Batch Normalization 
        x = keras.layers.BatchNormalization(
#             name=f'stage{stage_num}_batch2'
        )(x)

        # Conv2D
        y = keras.layers.Conv2D(
            filters=channel*4,
            kernel_size=(1,1),
            strides=stride,
            kernel_initializer='he_normal',
            padding='same'
#             name=f'stage{stage_num}_conv2'
        )(input_layer)

        # Batch Normalization 
        y = keras.layers.BatchNormalization(
#             name=f'stage{stage_num}_batch2'
        )(y)

        # Add
        x = keras.layers.Add()([x, y])

        # Activation
        x = keras.layers.Activation('relu')(x)      
    
    return x

#### ResNet 50 구성

In [7]:
def resnet_50(input_shape=(32,32,3),
              channel_list=[64,64,64,128,128,128,128,256,256,256,256,256,256,512,512,512],
              first_list=[True,False,False,True,False,False,False,True,False,False,False,False,False,True,False,False],
              num_classes=2):
    
    input_layer = keras.layers.Input(shape=input_shape)  # input layer를 만들어둡니다.
        

    # Conv1
    x = keras.layers.Conv2D(
        filters=64,
        kernel_size=(7,7),
        strides=2,
        kernel_initializer='he_normal',
        padding='same'
    )(input_layer)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = keras.layers.MaxPooling2D(
             pool_size=(2, 2),
             strides=2
    )(x)
    
    
    for i, (channel, first) in enumerate(zip(channel_list, first_list)): 
        x = ResNet50_Block(
            x,
            channel=channel,
            first = first
        )

        
    output = keras.layers.AveragePooling2D(
             pool_size=(2, 2),
             strides=2,
             padding='same'
             
    )(x)
    
    output = keras.layers.Flatten()(output)
    output = keras.layers.Dense(num_classes, activation='softmax')(output)

    
    model = keras.Model(
            inputs = input_layer,
            outputs = output)
    

    
    return model

### 2) Plain 50 구성

#### Plain 50 Block 구성하기

In [8]:
def plain50_Block(input_layer,
                 channel=64,
#                  stage_num=1,
                 first=False):
    
    # 입력 레이어
    x = input_layer
    
    if not first:
    
        # Conv2D
        x = keras.layers.Conv2D(
            filters=channel,
            kernel_size=(1,1),
            kernel_initializer='he_normal',
            padding='same'
#             name=f'stage{stage_num}_conv1'
        )(x)
    
        # Batch Normalization 
        x = keras.layers.BatchNormalization(
#             name=f'stage{stage_num}_batch1'
        )(x)

        # Activation
        x = keras.layers.Activation('relu')(x)

        # Conv2D
        x = keras.layers.Conv2D(
            filters=channel,
            kernel_size=(3,3),
            kernel_initializer='he_normal',
            padding='same'
#             name=f'stage{stage_num}_conv1'
        )(x)
    
        # Batch Normalization 
        x = keras.layers.BatchNormalization(
#             name=f'stage{stage_num}_batch1'
        )(x)

        # Activation
        x = keras.layers.Activation('relu')(x)        

        # Conv2D
        x = keras.layers.Conv2D(
            filters=channel*4,
            kernel_size=(1,1),
            kernel_initializer='he_normal',
            padding='same'
#             name=f'stage{stage_num}_conv2'
        )(x)

        # Batch Normalization 
        x = keras.layers.BatchNormalization(
#             name=f'stage{stage_num}_batch2'
        )(x)

        # Activation
        x = keras.layers.Activation('relu')(x)
    
    else:
        
        if channel == 64: 
            stride=1 
        else: 
            stride=2
            # Conv2D
        x = keras.layers.Conv2D(
            filters=channel,
            kernel_size=(1,1),
            strides=stride,
            kernel_initializer='he_normal',
            padding='same'
#           name=f'stage{stage_num}_conv1'
        )(x)

        # Batch Normalization 
        x = keras.layers.BatchNormalization(
#             name=f'stage{stage_num}_batch1'
        )(x)

        # Activation
        x = keras.layers.Activation('relu')(x)

        # Conv2D
        x = keras.layers.Conv2D(
            filters=channel,
            kernel_size=(3,3),
            kernel_initializer='he_normal',
            padding='same'
#             name=f'stage{stage_num}_conv1'
        )(x)

        # Batch Normalization 
        x = keras.layers.BatchNormalization(
#             name=f'stage{stage_num}_batch1'
        )(x)

        # Activation
        x = keras.layers.Activation('relu')(x)        

        # Conv2D
        x = keras.layers.Conv2D(
            filters=channel*4,
            kernel_size=(1,1),
            kernel_initializer='he_normal',
            padding='same'
#             name=f'stage{stage_num}_conv2'
        )(x)

        # Batch Normalization 
        x = keras.layers.BatchNormalization(
#             name=f'stage{stage_num}_batch2'
        )(x)

        # Activation
        x = keras.layers.Activation('relu')(x)      
    
    return x

#### Plain 50 함수 구성

In [9]:
def plain_50(input_shape=(32,32,3),
             channel_list=[64,64,64,128,128,128,128,256,256,256,256,256,256,512,512,512],
             first_list=[True,False,False,True,False,False,False,True,False,False,False,False,False,True,False,False],
             num_classes=2):
    
    input_layer = keras.layers.Input(shape=input_shape)  # input layer를 만들어둡니다.
        

    # Conv1
    x = keras.layers.Conv2D(
        filters=64,
        kernel_size=(7,7),
        strides=2,
        kernel_initializer='he_normal',
        padding='same'
    )(input_layer)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = keras.layers.MaxPooling2D(
             pool_size=(2, 2),
             strides=2
    )(x)


    
    for i, (channel, first) in enumerate(zip(channel_list, first_list)): 
        x = plain50_Block(
            x,
            channel=channel,
            first = first
        )

        
    output = keras.layers.AveragePooling2D(
             pool_size=(2, 2),
             strides=2,
             padding='same'
             
    )(x)
    
    
    output = keras.layers.Flatten()(output)
    output = keras.layers.Dense(num_classes, activation='softmax')(output)

    
    model = keras.Model(
            inputs = input_layer,
            outputs = output)

    
    return model

# 최종 ResNet, Plain 함수 사용하기

### build_resnet 함수 정의

In [10]:
def build_resnet(input_shape=(224,224,3),
                 is_50=False):       
   
    if not is_50:
        model = resnet_34(input_shape)
        
    
    else:
        model = resnet_50(input_shape)
    
    return model

#### build_resnet 함수로 ResNet 34 만들기

In [11]:
resnet_34 = build_resnet(is_50=False)

resnet_34.summary() 

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 112, 112, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 112, 112, 64) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

#### build_resnet 함수로 ResNet 50 만들기

In [12]:
resnet_50 = build_resnet(is_50=True)

resnet_50.summary() 

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_36 (Conv2D)              (None, 112, 112, 64) 9472        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_36 (BatchNo (None, 112, 112, 64) 256         conv2d_36[0][0]                  
__________________________________________________________________________________________________
activation_33 (Activation)      (None, 112, 112, 64) 0           batch_normalization_36[0][0]     
____________________________________________________________________________________________

### build_plain 함수 정의

In [13]:
def build_plain(input_shape=(224,224,3),
                is_50=False):       
   
    if not is_50:
        model = plain_34(input_shape)
        
    
    else:
        model = plain_50(input_shape)
    
    return model

#### build_plain 함수로 plain 34 함수 만들기

In [14]:
plain_34 = build_plain(is_50=False)

plain_34.summary() 

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_89 (Conv2D)           (None, 112, 112, 64)      9472      
_________________________________________________________________
batch_normalization_89 (Batc (None, 112, 112, 64)      256       
_________________________________________________________________
activation_82 (Activation)   (None, 112, 112, 64)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
block0_conv0 (Conv2D)        (None, 56, 56, 64)        36928     
_________________________________________________________________
block0_batch0 (BatchNormaliz (None, 56, 56, 64)        256 

#### build_plain 함수로 plain 50 함수 만들기

In [15]:
plain_50 = build_plain(is_50=True)

plain_50.summary() 

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_90 (Conv2D)           (None, 112, 112, 64)      9472      
_________________________________________________________________
batch_normalization_90 (Batc (None, 112, 112, 64)      256       
_________________________________________________________________
activation_83 (Activation)   (None, 112, 112, 64)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_91 (Conv2D)           (None, 56, 56, 64)        4160      
_________________________________________________________________
batch_normalization_91 (Batc (None, 56, 56, 64)        256 

## 데이터 불러오기

#### cats_vs_dogs

In [16]:
import urllib3
urllib3.disable_warnings()
dataset_name = 'cats_vs_dogs'
ds_train, ds_info_train = tfds.load(name=dataset_name, split='train[:80%]', as_supervised=True, with_info=True)
ds_valid, ds_info_valid = tfds.load(name=dataset_name, split='train[80%:]', as_supervised=True, with_info=True)

#### 전처리 함수

In [17]:
def preprocess(features):
    print(features['image'], features['label'])
    image = tf.image.resize(features['image'], [224,224])
    image = tf.divide(image, 255)
    print(image)
    label = features['label']
    print(label)
    return image, tf.cast(label, tf.float32)

#### normalize와 resize함수

In [18]:
def normalize_and_resize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    image = tf.image.resize(image, [224, 224])
    return tf.cast(image, tf.float32) / 255., label

#### 전체 적용하는 함수

In [19]:
def apply_normalize_on_dataset(ds, is_test=False, batch_size=16):
    ds = ds.map(
        normalize_and_resize_img, 
        num_parallel_calls=1
    )
    ds = ds.batch(batch_size)
    if not is_test:
        ds = ds.repeat()
        ds = ds.shuffle(200)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

In [20]:
print(ds_info_train.features)
print(ds_info_valid.features)

FeaturesDict({
    'image': Image(shape=(None, None, 3), dtype=tf.uint8),
    'image/filename': Text(shape=(), dtype=tf.string),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
})
FeaturesDict({
    'image': Image(shape=(None, None, 3), dtype=tf.uint8),
    'image/filename': Text(shape=(), dtype=tf.string),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
})


#### 하이퍼 파라미터 조정 (batch_size, epoch)

In [21]:
BATCH_SIZE = 64
EPOCH = 20

#### train와 valid에 normalize 적용

In [22]:
ds_train = apply_normalize_on_dataset(ds_train, batch_size=BATCH_SIZE)
ds_valid = apply_normalize_on_dataset(ds_valid, batch_size=BATCH_SIZE)

#### resnet_34

In [23]:
resnet_34.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

history_r34 = resnet_34.fit(
    ds_train,
    steps_per_epoch=int(ds_info_train.splits['train[:80%]'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info_valid.splits['train[80%:]'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_valid,
    verbose=1,
    use_multiprocessing=True,
)

Epoch 1/50
290/290 [==============================] - 253s 756ms/step - loss: 0.8801 - accuracy: 0.5462 - val_loss: 0.7255 - val_accuracy: 0.5833
Epoch 2/50
290/290 [==============================] - 236s 816ms/step - loss: 0.7556 - accuracy: 0.6120 - val_loss: 0.7316 - val_accuracy: 0.6342
Epoch 3/50
290/290 [==============================] - 216s 746ms/step - loss: 0.6598 - accuracy: 0.6661 - val_loss: 0.6902 - val_accuracy: 0.6532
Epoch 4/50
290/290 [==============================] - 217s 748ms/step - loss: 0.5802 - accuracy: 0.7111 - val_loss: 0.6715 - val_accuracy: 0.6861
Epoch 5/50
290/290 [==============================] - 217s 748ms/step - loss: 0.5207 - accuracy: 0.7487 - val_loss: 0.7325 - val_accuracy: 0.6645
Epoch 6/50
290/290 [==============================] - 217s 748ms/step - loss: 0.4458 - accuracy: 0.7916 - val_loss: 0.6376 - val_accuracy: 0.7092
Epoch 7/50
290/290 [==============================] - 217s 747ms/step - loss: 0.3564 - accuracy: 0.8405 - val_loss: 0.8398 -

#### resnet_50

In [24]:
resnet_50.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

history_r50 = resnet_50.fit(
    ds_train,
    steps_per_epoch=int(ds_info_train.splits['train[:80%]'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info_valid.splits['train[80%:]'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_valid,
    verbose=1,
    use_multiprocessing=True,
)

Epoch 1/50
290/290 [==============================] - 465s 1s/step - loss: 0.9884 - accuracy: 0.5307 - val_loss: 0.7644 - val_accuracy: 0.5333
Epoch 2/50
290/290 [==============================] - 397s 1s/step - loss: 0.8997 - accuracy: 0.5553 - val_loss: 1.0369 - val_accuracy: 0.5425
Epoch 3/50
290/290 [==============================] - 398s 1s/step - loss: 0.8497 - accuracy: 0.5766 - val_loss: 0.8539 - val_accuracy: 0.5918
Epoch 4/50
290/290 [==============================] - 398s 1s/step - loss: 0.8199 - accuracy: 0.5784 - val_loss: 0.9885 - val_accuracy: 0.5809
Epoch 5/50
290/290 [==============================] - 398s 1s/step - loss: 0.8004 - accuracy: 0.6028 - val_loss: 0.8750 - val_accuracy: 0.5994
Epoch 6/50
290/290 [==============================] - 398s 1s/step - loss: 0.7750 - accuracy: 0.6145 - val_loss: 0.8942 - val_accuracy: 0.5935
Epoch 7/50
290/290 [==============================] - 398s 1s/step - loss: 0.7315 - accuracy: 0.6324 - val_loss: 0.7606 - val_accuracy: 0.6310

KeyboardInterrupt: 

#### plain_34

In [ ]:
plain_34.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

history_p34 = plain_34.fit(
    ds_train,
    steps_per_epoch=int(ds_info_train.splits['train[:80%]'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info_valid.splits['train[80%:]'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_valid,
    verbose=1,
    use_multiprocessing=True,
)

#### plain_50

In [ ]:
plain_50.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

history_p50 = plain_50.fit(
    ds_train,
    steps_per_epoch=int(ds_info_train.splits['train[:80%]'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info_valid.splits['train[80%:]'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_valid,
    verbose=1,
    use_multiprocessing=True,
)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history_r34.history['loss'], 'r')
plt.plot(history_r50.history['loss'], 'b')
plt.plot(history_p34.history['loss'], 'g')
plt.plot(history_p50.history['loss'], 'y')
plt.title('Model training loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['resnet_34', 'resnet_50', 'plain_34', 'plain_50'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history_r34.history['val_accuracy'], 'r')
plt.plot(history_r50.history['val_accuracy'], 'b')
plt.plot(history_p34.history['val_accuracy'], 'g')
plt.plot(history_p50.history['val_accuracy'], 'y')
plt.title('Model validation accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['resnet_34', 'resnet_50', 'plain_34', 'plain_50'], loc='upper left')
plt.show()